In [1]:
from Bio import SeqIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
import os
import sys
sys.path.append('../../code')
import visualize
from crispr import CasTemplate
from pipeline import NDNFPipeline

In [3]:
# Path to a collection of HIV sequences to use as the nomination set.
NOMINATE_FASTA = '../../data/nominate.fasta'

# Path to a collection of HIV sequences to use as the validation set.
VALIDATE_FASTA = '../../data/test.fasta'

# Path to a single HIV sequence to use as the common reference position set.
REFERENCE_FASTA = '../../data/hxb2.fasta'

# Path to an off-target reference genome.
OFFTARGET_REF = '../../data/GRCh38.fna'

# The number of protospacers to extract from the nomination set.
TOP_N_KMERS = 20_000

# The target number of protospacers after diversification.
DIVERSITY_TARGET = TOP_N_KMERS*2

MISMATCH = 3
BROAD_SPECTRUM = 0.75
MAX_OFF = 0

In [4]:

cas_templates = [CasTemplate(pam_motif = 'NNNNCRAA', typ='II', 
                             protospacer_length=20, name='GeoCas9'),
                 
                 CasTemplate(pam_motif = 'NNGYRA', typ='II', 
                             protospacer_length=20, name='SpaCas9'),
                 
                 CasTemplate(pam_motif = 'TA', typ='V', 
                             protospacer_length=20, name='CasY'),
                 
                 CasTemplate(pam_motif = 'TTV', typ='V', 
                             protospacer_length=20, name='FnCas12a'),
                 
                 CasTemplate(pam_motif = 'RTR', typ='V', 
                             protospacer_length=20, name='Cas12h1'),
                 
                 CasTemplate(pam_motif = 'TBN', typ='V', 
                             protospacer_length=20, name='Cas12j'),
                 
                 CasTemplate(pam_motif = 'CCY', typ='V', 
                             protospacer_length=20, name='Asp2Cas12l'),
                 
                 CasTemplate(pam_motif = 'TTTN', typ='V', 
                             protospacer_length=20, name='AsCas12a'),
                 
                 CasTemplate(pam_motif = 'TTTR', typ='V', 
                             protospacer_length=20, name='Un1Cas12f1'),
                 
                  CasTemplate(pam_motif = 'TTN', typ='V', 
                             protospacer_length=20, name='AacCas12b'),
                 
                 CasTemplate(pam_motif = 'TN', typ='V', 
                             protospacer_length=20, name='Cas12c2'),
                 
                 CasTemplate(pam_motif = 'TTCN', typ='V', 
                             protospacer_length=20, name='Cas12e'),
                 
                 CasTemplate(pam_motif = 'TA', typ='V', 
                             protospacer_length=20, name='Cas12d'),
                 
                 CasTemplate(pam_motif = 'TG', typ='V', 
                             protospacer_length=20, name='OspCas12c'),
                 
                 CasTemplate(pam_motif = 'YTTN', typ='V', 
                             protospacer_length=20, name='ErCas12a'),
                 
                 CasTemplate(pam_motif = 'ATTN', typ='V', 
                             protospacer_length=20, name='BthCas12b'),
                 
                 CasTemplate(pam_motif = 'TTCN', typ='V', 
                             protospacer_length=20, name='Cas12e'),
                 
                 CasTemplate(pam_motif = 'RTR', typ='V', 
                             protospacer_length=20, name='Cas12h1'),
                 
                 CasTemplate(pam_motif = 'CCY', typ='V', 
                             protospacer_length=20, name='ASp2Cas12l'),
                 
                 CasTemplate(pam_motif = 'YG', typ='II', 
                             protospacer_length=20, name='FnCas9-RHA'),
                 CasTemplate(pam_motif = 'NGN', typ='II', 
                             protospacer_length=20, name='xCas'),
                 CasTemplate(pam_motif = 'NNGYRA', typ='II', 
                             protospacer_length=20, name='spaCas9'),
                 CasTemplate(pam_motif = 'NNG', typ='II', 
                             protospacer_length=20, name='ScCas9'),
                 
                 CasTemplate(pam_motif = 'NNAGAAW', typ='II', 
                             protospacer_length=20, name='St1Cas9'),
                 CasTemplate(pam_motif = 'NNGG', typ='II', 
                             protospacer_length=20, name='SauriCas9'),
                 CasTemplate(pam_motif = 'NNRG', typ='II', 
                             protospacer_length=20, name='SauriCas9-KKH'),
                 
                 CasTemplate(pam_motif = 'NNNRYAC', typ='II', 
                             protospacer_length=20, name='CjeCas9'),
                 
                 CasTemplate(pam_motif = 'NNNNGATT', typ='II', 
                             protospacer_length=20, name='Nme1Cas9'),
                ]
                 

In [5]:
with open(NOMINATE_FASTA) as handle:
    nomination_seqs = list(SeqIO.parse(handle, 'fasta'))
num_nom = len(nomination_seqs)
len_nom = sum(len(seqR.seq) for seqR in nomination_seqs)/1_000_000
print(f'Loading {num_nom} sequences with a total of {len_nom:0.2f} Mbp for nomination.')

Loading 3228 sequences with a total of 28.73 Mbp for nomination.


In [6]:
MUT_DATA = pd.read_csv('../../data/mutation_scores.csv')

def rc_scores(start, width=20):
    
    mask = (MUT_DATA['Position'] <= (start+width)) & (MUT_DATA['Position'] >= (start-width))
    mean_rc = MUT_DATA.loc[mask, 'RC Index'].mean()
    
    return mean_rc

def lethal_count(start, width=20):
    
    mask = (MUT_DATA['Position'] <= (start+width)) & (MUT_DATA['Position'] >= (start-width))
    num_lethal = (MUT_DATA.loc[mask, 'Effect'] == "lethal").sum()
    
    return num_lethal

def process_cas(cas):
    
    nominated, protospacer_counts = NDNFPipeline.nominate(cas, nomination_seqs, 
                                                          top_n=TOP_N_KMERS, 
                                                          return_counts=True)
    diversed = nominated.diversify(DIVERSITY_TARGET)
    narrowed, protospacer_hist_freq = diversed.narrow(VALIDATE_FASTA, min_rate=0.5, return_freqs=True,
                                                      mismatch=MISMATCH)
    good_hits, off_target_counts = narrowed.filter(OFFTARGET_REF, 
                                               mismatch = MISMATCH, 
                                               return_counts=True)
    ref_diversed = diversed.reference_map(REFERENCE_FASTA, mismatch=3, enforce_pam=False)
    ref_diversed['on_target_freq'] = ref_diversed['protospacer'].map(protospacer_hist_freq.get)
    ref_diversed['off_target_hits'] = ref_diversed['protospacer'].map(off_target_counts.get)
    ref_diversed['num_lethal'] = ref_diversed['start'].map(lethal_count)
    ref_diversed['mean_rc'] = ref_diversed['start'].map(rc_scores)
    ref_diversed['cas'] = cas.name
    ref_diversed['pam_motif'] = cas.pam_motif
    return ref_diversed
    

In [7]:
for cas in cas_templates:
    print('Processing', cas.name)
    fname = f'{cas.name}.info.csv'
    if not os.path.exists(fname):
        process_cas(cas).to_csv(fname, index=False)


Processing GeoCas9
Processing SpaCas9


100%|██████████| 40000/40000 [06:52<00:00, 96.89it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
47.058 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
40.3734 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
34.6305 seconds elapsed.


Processing CasY


100%|██████████| 40000/40000 [06:52<00:00, 97.05it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
181.014 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
1311.24 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
38.0643 seconds elapsed.


Processing FnCas12a
Processing Cas12h1
Processing Cas12j


...

100%|██████████| 40000/40000 [06:51<00:00, 97.27it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
294.507 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
5487.83 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
38.1507 seconds elapsed.


Processing CjeCas9


100%|██████████| 40000/40000 [07:18<00:00, 91.20it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
51.7116 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
48.0981 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
34.4264 seconds elapsed.


Processing Nme1Cas9


100%|██████████| 40000/40000 [06:57<00:00, 95.80it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
35.5085 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
28.1915 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
35.9515 seconds elapsed.
